In [9]:
import torch
from torch import nn
import torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import random
import json
from typing import List, Dict, Tuple

class ThreatGenerator(nn.Module):
    def __init__(self):
        super(ThreatGenerator, self).__init__()
        self.gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def generate_threat(self, prompt: str, max_length: int = 100) -> str:
        inputs = self.tokenizer.encode(prompt, return_tensors='pt')
        outputs = self.gpt2.generate(
            inputs,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            pad_token_id=self.tokenizer.eos_token_id  # Set pad token
        )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

class ResponseGenerator(nn.Module):
    def __init__(self):
        super(ResponseGenerator, self).__init__()
        self.gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def generate_response(self, threat: str, max_length: int = 150) -> str:
        prompt = f"Threat: {threat}\nResponse:"
        inputs = self.tokenizer.encode(prompt, return_tensors='pt')
        outputs = self.gpt2.generate(
            inputs,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            pad_token_id=self.tokenizer.eos_token_id  # Set pad token
        )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

class ThreatClassifier(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int):
        super(ThreatClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return F.softmax(self.fc2(x), dim=1)

class DynamicThreatSimulator:
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.gpt2 = GPT2LMHeadModel.from_pretrained('gpt2').to(self.device)
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.threat_generator = ThreatGenerator().to(self.device)
        self.response_generator = ResponseGenerator().to(self.device)
        self.threat_classifier = ThreatClassifier(input_dim=768, hidden_dim=500, output_dim=5).to(self.device)
        self.threat_categories = ['malware', 'phishing', 'ddos', 'data_breach', 'insider_threat']
        self.report_data = []  # Store the reports for future export

    def classify_threat(self, threat: str) -> Tuple[str, float]:
        inputs = self.tokenizer(threat, return_tensors='pt', max_length=512, truncation=True, padding='max_length').to(self.device)
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']

        with torch.no_grad():
            outputs = self.gpt2.transformer(input_ids, attention_mask=attention_mask)
            hidden_states = outputs.last_hidden_state

        pooled_output = hidden_states.mean(dim=1)

        with torch.no_grad():
            classification_logits = self.threat_classifier(pooled_output)

        predicted_category = self.threat_categories[classification_logits.argmax().item()]
        severity_score = classification_logits.max().item() * 5  # scale severity to 5

        return predicted_category, severity_score

    def generate_response_strategy(self, threat: str) -> str:
        return self.response_generator.generate_response(threat)

    def report_to_cybersecurity(self, activity: str, severity: float):
        print(f"\n🚨 Reporting to cybersecurity: Suspicious activity '{activity}' has a severity of {severity:.2f}")

    def suggest_fixes(self, category: str) -> str:
        fix_suggestions = {
            'malware': [
                "Update your anti-virus software and run a full scan.",
                "Quarantine affected systems.",
                "Monitor traffic for unusual activity."
            ],
            'phishing': [
                "Enable multi-factor authentication (MFA).",
                "Train employees to spot phishing emails.",
                "Block phishing domains."
            ],
            'ddos': [
                "Use DDoS protection services.",
                "Limit server connections per IP.",
                "Monitor network traffic and set alerts."
            ],
            'data_breach': [
                "Change all affected credentials.",
                "Audit access logs and notify impacted users.",
                "Strengthen encryption and access controls."
            ],
            'insider_threat': [
                "Review access logs.",
                "Restrict access to sensitive data.",
                "Implement activity monitoring tools."
            ]
        }
        return "\n".join(fix_suggestions.get(category, ["No specific fix available."]))

    def process_suspicious_activities(self, activities: List[Dict[str, str]]):
        print("\n🌐 Processing suspicious activities...")
        for i, activity_data in enumerate(activities):
            activity = activity_data["description"]
            category, severity = self.classify_threat(activity)
            print(f"\n🚩 Suspicious Activity {i+1}: {activity}")
            print(f"  - Classified as: {category}")
            print(f"  - Severity: {'⚠️ ' if severity > 3 else 'ℹ️ '}{severity:.2f}")

            response = self.generate_response_strategy(activity)
            print(f"  - Response: {response}")

            fixes = self.suggest_fixes(category)
            print(f"  - Suggested Fixes:\n{fixes}")

            if severity > 3:
                self.report_to_cybersecurity(activity, severity)

            # Append data to report for logging
            self.report_data.append({
                "activity": activity,
                "category": category,
                "severity": severity,
                "response": response,
                "fixes": fixes
            })

        print("\n✔️ All activities processed.")

    def export_report(self, file_name="cybersecurity_report.json"):
        with open(file_name, 'w') as f:
            json.dump(self.report_data, f, indent=4)
        print(f"\n📊 Report exported to {file_name}.")

    def ask_for_input(self) -> List[Dict[str, str]]:
        activities = []
        print("\n🔍 Enter suspicious activities. Type 'q' to finish.")
        while True:
            activity = input("Enter a suspicious activity description: ")
            if activity.lower() == 'q':
                break
            activities.append({"description": activity})
        return activities

def main():
    simulator = DynamicThreatSimulator()

    print("\n🔥 Welcome to the Advanced Cybersecurity Threat Simulator!")
    print("This tool helps you classify and mitigate suspicious activities in real-time.")

    while True:
        print("\nMenu:")
        print("1. Simulate a threat scenario")
        print("2. Enter suspicious activities")
        print("3. Export report")
        print("4. Exit")
        choice = input("\nChoose an option (1-4): ")

        if choice == '1':
            scenario = simulator.ask_for_input()
            if scenario:
                simulator.process_suspicious_activities(scenario)
        elif choice == '2':
            simulator.process_suspicious_activities(simulator.ask_for_input())
        elif choice == '3':
            simulator.export_report()
        elif choice == '4':
            print("\n👋 Exiting... Stay safe!")
            break
        else:
            print("Invalid choice! Please enter a valid option.")

if __name__ == "__main__":
    main()



🔥 Welcome to the Advanced Cybersecurity Threat Simulator!
This tool helps you classify and mitigate suspicious activities in real-time.

Menu:
1. Simulate a threat scenario
2. Enter suspicious activities
3. Export report
4. Exit

Choose an option (1-4): 1

🔍 Enter suspicious activities. Type 'q' to finish.
Enter a suspicious activity description: spam emails
Enter a suspicious activity description: junk calls
Enter a suspicious activity description: q

🌐 Processing suspicious activities...

🚩 Suspicious Activity 1: spam emails
  - Classified as: malware
  - Severity: ℹ️ 2.00
  - Response: Threat: spam emails
Response: no spam
The following is a list of all the spam email addresses that have been sent to you by the following email address:

Email Address: Your email is not valid. Please try again later.
. Your e-mail address is invalid. You can try another email at any time.
  - Suggested Fixes:
Update your anti-virus software and run a full scan.
Quarantine affected systems.
Monitor t